In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from math import *
import scipy.stats
from pandas import *
#import scipy.integrate as integrate
from scipy.integrate import quad
import seaborn as sns
import scipy as s
import warnings
warnings.filterwarnings("ignore")
from mpl_toolkits import mplot3d
from tqdm import tqdm
import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.graph_objects as go
from scipy import linalg 
#pandas.set_option('display.max_rows',10000)

In [2]:
import time as tp

In [13]:
# Define function to calculate jump distribution for integral terms 

def jump_distribution_full(x_value,mu,sigma_jump,step_x):
    lower = x_value - step_x/2.0
    upper = x_value + step_x/2.0
    #print(lower,upper)
    def normal_distribution_function(x):
        value = scipy.stats.norm.pdf(x,mu,sigma_jump)
        return value
    res, err = quad(normal_distribution_function, lower, upper)
    return res  #output is a vector of the average densities around the regions of each value in the vector x

In [14]:
# Coefficients as obtained from BTCS discretization scheme

def a_coeff(x_value, sigma, kappa, theta):
    coefficient = (sigma**2)/(dx**2)       
    return coefficient

def b_coeff(x_value, sigma, kappa, theta):
    coefficient_2 = 0.5*(sigma**2)*(1/dx**2) + 0.5*(1/dx)*kappa*(theta-x_value)
    return coefficient_2

def c_coeff(x_value, sigma, kappa, theta):
    coefficient_3 = 0.5*(sigma**2)*(1/dx**2) - 0.5*(1/dx)*kappa*(theta-x_value)
    return coefficient_3   

In [15]:
# Construction of M matrix containing the coefficients of the solution at time t = q+1 in the implicit scheme 

def M_matrix(x, sigma, kappa, theta):
    M = np.zeros(shape=(len(x),len(x)))
    for i in range(0,len(x)):
        if x[i] <= 0 or x[i] >= L:
            M[i,i] = 1.0
        else:
            #print(x[i])
            a_coefficient = a_coeff(x[i], sigma, kappa, theta)
            b_coefficient = b_coeff(x[i], sigma, kappa, theta)
            c_coefficient = c_coeff(x[i], sigma, kappa, theta)
            if a_coefficient < 0 or b_coefficient < 0 or c_coefficient <0:
                print('Error')
            M[i,(i-1):(i+2)] = [-dt*c_coefficient, 1+dt*a_coefficient, -dt*b_coefficient]

    return M

In [16]:
# Calculation of integral term in the PIDE

def jump_integral(x, mu, N, sigma_jump, step_x):
    integral_list = []
    positions = np.arange(int(-N/2+1),int(N/2),1)
    for j in positions: 
        x_jump = step_x*j
        integral = jump_distribution_full(x_jump, mu, sigma_jump, step_x)
        #print(x_jump, integral)
        integral_list.append(integral)

    return sum(integral_list)

In [17]:
# Construction of matrix containing the coefficients of the solution at time t = q in the implicit scheme 


def N_matrix(x, poisson_rate, N, mu, sigma_jump, step_x):
    N_m = np.zeros(shape=(len(x),len(x)))
    positions = np.arange(int(-N/2+1),int(N/2),1)
    jump_result_list = []
    for i in range(0,(len(x))):
        integral_list = []
        if x[i] > 0.0 and x[i] < L:
            for j in positions: 
                x_jump = step_x*j
                if (i+j) >= 0 and (i+j) <= (len(x)-1):
                    integral = jump_distribution_full(x_jump, mu, sigma_jump, step_x)
                    integral_list.append(integral)
                    N_m[i,i+j] = poisson_rate*dt*integral
                    
            jump_integral = sum(integral_list)
            N_m[i,i] = N_m[i,i]+ (1 - poisson_rate*dt*jump_integral)
            
            jump_result_list.append(jump_integral)
            if 1-poisson_rate*dt*jump_integral < 0:
                print('Error in integral approximation')
                
    print(f'Min: {min(jump_result_list,  default="EMPTY")} Max: {max(jump_result_list,  default="EMPTY")}')
            
    return N_m

In [18]:
# Define x grid by extending the boundary to account for the jump term

x0 = -10.0
xn = 10.0
L = 8.0
xsteps = 1000
dx = (xn-x0)/xsteps
x = np.arange(x0, xn+dx, dx) #last x value is x = 99 

In [19]:
# Define the t grid

t0 = 0.0
tn = 1.0
tsteps = 1000
dt = (tn-t0)/tsteps
t = np.arange(t0, tn+dt, dt) #last time value is t = 999.9
r = dt / (dx**2) # ensure r < 1
print(dt)
#print(t,r,len(t),tsteps,dt)

0.001


In [20]:
# Example 

sigma = 2.0 
kappa = 0.5 
theta = 3.5
mu_jump = 0.0
sigma_jump = 0.2
rate = 1.0

In [21]:
# Example - applying the FD iterations

t_zero = tp.time()

sigma_choice = sigma
kappa_choice = kappa
theta_choice = theta
N_choice = 150
mu_choice = mu_jump
sigma_jump_choice = sigma_jump
poisson_rate = rate
int_approx = jump_integral(x, mu_choice, N_choice, sigma_jump_choice, dx)
print(int_approx)


b = np.zeros(len(x))
b[x >= L] = 1.0
phi_matrix = np.zeros(shape=(len(x), len(t)))
phi_matrix[:,0] = 1.0 * (x>0.0)
phi_matrix[len(x)-1,:] = 1.0


M = M_matrix(x, sigma = sigma_choice, kappa = kappa_choice, theta = theta_choice)
N_mat = N_matrix(x, poisson_rate = poisson_rate, N = N_choice, 
                 mu = mu_choice, sigma_jump = sigma_jump_choice, step_x=dx)


if (sigma_choice**2 < np.max(np.abs(dx*kappa_choice*(theta_choice-x)))) or ((poisson_rate*int_approx*dt > 1)):
    print('Stability conditions may not be satisfied')
    
for time in tqdm(range(1, len(t))): 
    phi_matrix[:,time] = np.linalg.inv(M).dot(N_mat.dot(phi_matrix[:,time-1]) + b)

t_final = tp.time()
print(t_zero-t_final)

0.999999999999907


  0%|          | 1/1000 [00:00<02:05,  7.94it/s]

Min: 0.999999999999907 Max: 0.999999999999907


100%|██████████| 1000/1000 [01:54<00:00,  8.75it/s]

-457.8346438407898


In [8]:
# Keep "main" solution within the domain

accept_x = np.where((x>=0) & (x<=L))[0]
#accept_x

In [10]:
def survival(space,time):
    return phi_matrix[space, time]

In [ ]:
# Plot main solution

X = accept_x #np.arange(50,65,1)
T = np.arange(0,len(t),1)
X, T = np.meshgrid(X, T)
Phi = survival(X,T)

data=go.Surface(z=Phi, x=x0+dx*X, y=t0+dt*T)

layout = go.Layout(scene = dict(
                    xaxis_title='Initial Position',
                    yaxis_title='Time until maturity',
                    zaxis_title='Survival Probability'))

fig = go.Figure(data=[data], layout=layout)
pyoff.plot(fig)
#fig.show()